In [ ]:
import torch 
from utils import * 
from Kalman import *
from KalmanNet import *

torch.autograd.set_detect_anomaly(True)

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150

from sklearn.model_selection import train_test_split

In [ ]:
N = 5000
D = 3
M = 8
SNR = 20
array = ArrayModel(M, perturbation=0.0)

T = 40
Ts = 1
Ns = 200

batchSize = 128
lr = 1e-5
wd = 1e-9
nbEpoches = 10

In [ ]:
X, y = generate_dataset(N=N, D=D, T=T, Ts=Ts, Ns=Ns, array=array, SNR=SNR)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
net = KalmanNet(array=array, d=D, Ts=Ts, Ns=Ns, device=dev)
train_loader = DataLoader(DATA(X_train, y_train, Ns=net.Ns), batch_size=batchSize, shuffle=True)
opt = optim.Adam(params=net.parameters(), lr=lr, weight_decay=wd)

Loss = []

for i in range(nbEpoches):

    running_loss = 0.0
    for data in train_loader:
        input, target = data[0].to(net.device), data[1].to(net.device)
        opt.zero_grad()
        pred = net(input)
        loss = nn.MSELoss()(pred, target[:, 2:])
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        opt.step()
        running_loss += loss.item()
    Loss.append(running_loss/len(train_loader))
    print("Loss {} is {}".format(i, Loss[-1]))